This is a component of a larger project [Cat-A-Logger](https://github.com/screamatthewind/cat-a-logger) on github   
See this [Short Slide Presentation](https://github.com/screamatthewind/cat-a-logger/blob/main/Slide%20Presentation%20-%20Short.pdf)

In [1]:
# are we running locally or in kaggle?

import os

if os.environ.get('KAGGLE_KERNEL_RUN_TYPE','') == '':
    print("We are running code on Localhost")
    isLocalhost = True

else:
    print("We are running in Kaggle")
    isLocalhost = False

We are running code on Localhost


In [2]:
if isLocalhost:
    INPUT_FILES = './output/cropped-cats-and-dogs/*.jpg'
    
else:
    INPUT_FILES = '/kaggle/input/cropped-cats-and-dogs/*.jpg'

    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()

    USER_ID = user_secrets.get_secret("user-id")
    API_TOKEN = user_secrets.get_secret("api-token")
    
OUTPUT_DATASET_ID = 'augmented-cats-and-dogs'
OUTPUT_DATASET_NAME = 'Augmented Cats and Dogs'
OUTPUT_PATH = './output/augmented-cats-and-dogs'

NUM_AUGMENTATIONS = 10
INPUT_BATCH_SIZE = 100

# final image size
# Same size is used in Crop Cats and Dogs
X_SIZE = 224
Y_SIZE = 224

In [3]:
import os
if not os.path.exists(OUTPUT_PATH):
    os.makedirs(OUTPUT_PATH)

In [4]:
# if pad == True, maintain aspect ratio and pad images otherwise just rescale
def image_resize(image, x_size, y_size, pad):
    
    if pad:
        new_image = np.zeros((y_size, x_size, 3), np.uint8)
        new_image[:, 0:x_size] = (0, 255, 0) # (B, G, R) -- pure green padding

        w,h,c = image.shape
        
        if w > h:
            scale_factor = x_size/w
        else:
            scale_factor = y_size/h
            
        image = cv2.resize(image, (0,0), fx=scale_factor, fy=scale_factor)

        x_offset = int((x_size - image.shape[1])/2)
        y_offset = int((y_size - image.shape[0])/2)

        new_image[ y_offset:y_offset+image.shape[0], x_offset:x_offset+image.shape[1]] = image

        return new_image
    
    else:
        image = cv2.resize(image, (x_size, y_size))
        return image

This section inspired by: [Keras ImageDataGenerator and Data Augmentation](https://www.pyimagesearch.com/2019/07/08/keras-imagedatagenerator-and-data-augmentation/) by [Adrian Rosebrock](https://www.pyimagesearch.com/author/adrian/)


In [5]:
! pip install imutils

In [6]:
import os
import cv2
import time
import imageio
import numpy as np
import matplotlib.pyplot as plt

from imutils import paths

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical

In [7]:
# remove all masking artifacts from image
# very slow and inefficient, could be done better
from PIL import Image, ImageDraw

def mask_alpha(img):

    RED, GREEN, BLUE, ALPHA = (0, 1, 2, 3)

    x,y,c = img.shape
    alpha_channel = np.zeros([x,y])

    for i in range(x):

        for j in range(y):

            r=img[i][j][RED]
            g=img[i][j][GREEN]
            b=img[i][j][BLUE]

            if (r == 0.0 and g == 0.0 and b == 0.0) or (r == 0.0 and g == 1.0 and b == 0.0):
                alpha_channel[i][j] = 0.0
            else:
                alpha_channel[i][j] = 1.0

    # erode mask to get rid of more of the green screen
    kernel = np.ones((5,5), np.uint8)  
    alpha_channel = cv2.erode(alpha_channel, kernel, iterations=1)
    alpha_channel = np.expand_dims(alpha_channel,axis=2)

    img = np.concatenate((img, alpha_channel), axis=2)

    # crop image using alpha mask
    pil_img = Image.fromarray((img * 255).astype(np.uint8))
    mask = Image.new("RGBA", pil_img.size, (0, 0, 0, 0))
    bbox_image = Image.composite(pil_img, mask, pil_img)
    bbox = bbox_image.convert("RGBa").getbbox()
    pil_img = pil_img.crop(bbox)
    img = np.array(pil_img)

    return img

In [8]:
def augmentImages(data, filenames):

    # init augmentor
    aug = ImageDataGenerator(
        rotation_range=20,
        zoom_range=0.15,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.15,
        horizontal_flip=True,
        fill_mode="constant",
        cval = 0)

    file_num = 0

    # we need to break the loop by hand because the generator loops indefinitely
    batch_ctr = 0

    for images, labels in aug.flow(data, filenames, batch_size=NUM_AUGMENTATIONS):

        batch_ctr += 1
        if batch_ctr >= len(data):
            break

        for i in range(images.shape[0]):

            img=images[i]
            img = mask_alpha(img)

            file_num = file_num + 1
            filename = labels[i] + '-' + str(file_num) +  '.png'

            imageio.imwrite(OUTPUT_PATH + '/' + filename, img)
            print(filename)

In [9]:
def processBatch(imagePaths):

    data = []
    filenames = []
    
    # loop over the image paths
    for imagePath in imagePaths:

        label = imagePath.split(os.path.sep)[-2]
        image = cv2.imread(imagePath)
        image = image_resize(image, X_SIZE, Y_SIZE, True)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # switch the color oder

        # output original image as png with mask removed
        original_image = np.array(image, dtype="float") / 255.0
        original_image = mask_alpha(original_image)

        filename, file_extension = os.path.splitext(os.path.basename(imagePath))
        imageio.imwrite(OUTPUT_PATH + '/' + filename + '-original.png', original_image)
        print( filename + '-original.png')

        # update the data and labels lists, respectively
        data.append(image)
        filenames.append(filename)
        
        # convert the data into a NumPy array, then preprocess it by scaling all pixel intensities to the range [0, 1]
        np_data = np.array(data, dtype="float") / 255.0

        # encode the labels (which are currently strings) as integers and then one-hot encode them
        le = LabelEncoder()
        labels = le.fit_transform(filenames)

    # perform augmentation on batch of images
    augmentImages(np_data, filenames)
        

In [ ]:
import time
import glob

filenames = []
batch_ctr = 0

start_time = time.time()

# augment files in batches
for filename in glob.iglob(INPUT_FILES):

    filenames.append(filename)
    batch_ctr = batch_ctr + 1
    
    if (batch_ctr > INPUT_BATCH_SIZE):
        processBatch(filenames)
        filenames.clear()
        batch_ctr = 0

# process remaining files     
if len(filenames) > 0:
    processBatch(filenames)

run_time = time.time()-start_time
print('Done Augmenting Images - Total Time: {:.1f}'.format(run_time) + ' Secs')



cat.0-0-bw-original.png
cat.0-0-neg-original.png
cat.0-0-resized-bw-original.png
cat.0-0-resized-neg-original.png
cat.0-0-resized-original.png
cat.0-0-original.png
cat.1-4-bw-original.png
cat.1-4-neg-original.png
cat.1-4-resized-bw-original.png
cat.1-4-resized-neg-original.png
cat.1-4-resized-original.png
cat.1-4-original.png
cat.10-3-bw-original.png
cat.10-3-neg-original.png
cat.10-3-resized-bw-original.png
cat.10-3-resized-neg-original.png
cat.10-3-resized-original.png
cat.10-3-original.png
cat.100-1-bw-original.png
cat.100-1-neg-original.png
cat.100-1-resized-bw-original.png
cat.100-1-resized-neg-original.png
cat.100-1-resized-original.png
cat.100-1-original.png
cat.10000-4-bw-original.png
cat.10000-4-neg-original.png
cat.10000-4-resized-bw-original.png
cat.10000-4-resized-neg-original.png
cat.10000-4-resized-original.png
cat.10000-4-original.png
cat.10001-3-bw-original.png
cat.10001-3-neg-original.png
cat.10001-3-resized-bw-original.png
cat.10001-3-resized-neg-original.png
cat.1000

This section insipred by: [kaggle uploader[](http://)](https://www.kaggle.com/donkeys/kaggle-uploader)

In [ ]:
# Google Cloud Plaform for Kaggle(Beta) does not support /usr/lib modules at this time 
# Save Output Dataset

if isLocalhost == False:

    ! python -m pip install --index-url https://test.pypi.org/simple/ --no-deps kaggle_uploader-screamatthewind

    import time
    import os

    from kaggle_uploader import kaggle_uploader 

    print("Saving Images to Kaggle")
    start_time = time.time()

    # kaggle_secrets are not supported by Google Cloud Platform for Kaggle(Beta) at this time
    # from kaggle_secrets import UserSecretsClient
    # user_secrets = UserSecretsClient()
    # api_secret = user_secrets.get_secret("Crop Cats and Cogs YOLOv3")

    kaggle_uploader.resources = []
    kaggle_uploader.init_on_kaggle(USER_ID, API_TOKEN)
    kaggle_uploader.base_path = OUTPUT_PATH
    kaggle_uploader.title = OUTPUT_DATASET_NAME
    kaggle_uploader.dataset_id = OUTPUT_DATASET_ID
    kaggle_uploader.user_id = USER_ID

    for filename in os.listdir(kaggle_uploader.base_path):
        print(filename)
        kaggle_uploader.add_resource(filename, filename)

    kaggle_uploader.update("new version")

    run_time = time.time()-start_time
    print('Done Saving Images - Total Time: {:.1f}'.format(run_time) + ' Secs')

    # If you get an error during update, it is typically because of an invalid api key, bad username, 
    # or the dataset does not exist.  This code does not create datasets.  It updates existing ones

else:
    print("Done")